In [45]:
import boto3
import sagemaker
import pandas as pd

from glob import glob

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [42]:
training_job_name = 'tensorflow-training-2024-04-20-05-32-55-117'

In [13]:
!aws s3 cp s3://sagemaker-us-east-1-211125778552/tensorflow-training-2024-04-21-01-56-36-032/output/model.tar.gz ./model.tar.gz

download: s3://sagemaker-us-east-1-211125778552/tensorflow-training-2024-04-21-01-56-36-032/output/model.tar.gz to ./model.tar.gz


In [14]:
!mkdir -p ./model/
!tar -xvzf ./model.tar.gz -C ./model/

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
code/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/validation/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/validation/events.out.tfevents.1713665898.ip-10-2-64-28.ec2.internal.90.28606.v2
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/validation/events.out.tfevents.1713665659.ip-10-2-64-28.ec2.internal.90.25864.v2
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/train/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/train/events.out.tfevents.1713664780.ip-10-2-64-28.ec2.internal.profile-empty
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/train/plugins/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.c

In [15]:
!saved_model_cli show --all --dir ./model/tensorflow/saved_model/0/

2024-04-21 02:37:40.608942: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2024-04-21 02:37:40.608977: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 183)
        name: serving_defa

### Deploying to REST Endpoint

In [92]:
from sagemaker.tensorflow.model import TensorFlowModel
import time

In [93]:
timestamp = int(time.time())
tf_model_name = f"{training_job_name}-tf-{timestamp}"
tf_model_name

'tensorflow-training-2024-04-20-05-32-55-117-tf-1713674056'

In [94]:
tf_model = TensorFlowModel(
    name = tf_model_name,
    source_dir = 'src',
    entry_point = 'inference.py',
    model_data = f's3://{bucket}/{training_job_name}/output/model.tar.gz',
    role = role,
    framework_version = '2.3.1'   
)

In [102]:
tf_model.deploy(
    endpoint_name = tf_model_name,
    initial_instance_count = 1,
    instance_type = 'ml.c4.xlarge',
    wait = False
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Using already existing model: tensorflow-training-2024-04-20-05-32-55-117-tf-1713674056


ClientError: An error occurred (ValidationException) when calling the CreateEndpoint operation: Cannot create already existing endpoint "arn:aws:sagemaker:us-east-1:211125778552:endpoint/tensorflow-training-2024-04-20-05-32-55-117-tf-1713674056".

### DEMO: Predicting customer satisfaction on future comments

In [97]:
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer
import json

In [100]:
time.sleep(30)

In [99]:
predictor = TensorFlowPredictor(
    endpoint_name = tf_model_name,
    sagemaker_session = sess,
    model_name = "saved_model",
    serializer = JSONLinesSerializer(),
    deserializer = JSONLinesDeserializer(),
    content_type = 'application/jsonlines',
    accept_type = 'application/jsonlines'
)

content_type is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [113]:
review1 = '''I wholeheartedly recommend the RyanAir experience to anyone! 
The seats were comfortable, and everyone was so friendly.'''

review2 = '''I was not happy with how I was treated in this last flight. 
The in-flight entertainment consoles were broken, and I was treated rudely.'''

review3 = '''Overall, a nice journey. The seats were a little worn down, and I had to exchange seats,
 but I was given a discount on my tickets and overall am happy with my decision.'''

inputs = [{"features": [review1]}, {"features": [review2]}, {"features": [review3]}]

predicted_classes = predictor.predict(inputs)


for predicted_class in predicted_classes:
    print("Recommendation: {}".format(predicted_class['predicted_label']).replace('1', 'Not Recommended').replace('5', 'Recommended'))



Recommendation: Recommended
Recommendation: Not Recommended
Recommendation: Recommended
